<p> <center> <a href="../Start_here.ipynb">Home Page</a> </center> </p>

<div>
    <span style="float: left; width: 33%; text-align: left;"><a href="5.Measure_object_size_using_OpenCV.ipynb">Previous Notebook</a></span>
    <span style="float: left; width: 34%; text-align: center;">
        <a href="1.Data_labeling_and_preprocessing.ipynb">1</a>
        <a href="2.Object_detection_using_TAO_YOLOv4.ipynb">2</a>
        <a href="3.Model_deployment_with_Triton_Inference_Server.ipynb">3</a>
        <a href="4.Model_deployment_with_DeepStream.ipynb">4</a>
        <a href="5.Measure_object_size_using_OpenCV.ipynb">5</a>
        <a >6</a>
        <a href="7.Challenge_Triton.ipynb">7</a>
    </span>
    <span style="float: left; width: 33%; text-align: right;"><a href="7.Challenge_Triton.ipynb">Next Notebook</a></span>
</div>

# Exercise: model deployment with DeepStream

***

In this notebook, you will review the concepts learned in [4.Model_deployment_with_DeepStream.ipynb](4.Model_deployment_with_DeepStream.ipynb) while trying to deploy your TAO Toolkit model to DeepStream using Python bindings.

As an exercise, you are asked to re-implement the same 6-class object detection pipeline with a tracker that has been analyzed in the tutorial notebook. Here are the illustrations of the pipeline: remember that the secondary classifiers (highlighted in gray) are not to be implemented.

<img src="images/test2.png" width="1080">

Let us get started with the notebook. You will have to fill in the `COMPLETE THIS SECTION` parts of the code present in the notebook to complete the pipeline. Feel free to refer to the previous notebooks for the commands but make sure to grasp the most important underlying concepts.

## Building the pipeline


In [ ]:
# Import required libraries
import sys
sys.path.append("../source_code/N4")
from bus_call import bus_call
import gi
gi.require_version('Gst', '1.0')
from gi.repository import GLib, Gst
import configparser
import pyds
import time

############# ~~~~~~~ COMPLETE THIS SECTION ~~~~~~~ #############
# Define class labels
PGIE_CLASS_ID_FRESHAPPLE = 0
#
#
#
#
#
###################### ~~~~~~~ END ~~~~~~~ ######################

# Define input/output video files
INPUT_VIDEO_NAME  = "../source_code/N4/apples.h264" # Source: https://depositphotos.com
OUTPUT_VIDEO_NAME = "../source_code/challenge_deepstream/ds_out.mp4"

import os
if not os.path.exists("../source_code/challenge_deepstream"):
    !mkdir ../source_code/challenge_deepstream

First, we define a function `make_elm_or_print_err()` to create our elements and report any errors if the creation fails. Elements are created using the `Gst.ElementFactory.make()` function as part of Gstreamer library.

In [ ]:
# Make element or print error and any other detail
def make_elm_or_print_err(factoryname, name, printedname, detail=""):
    print("Creating", printedname)
    elm = Gst.ElementFactory.make(factoryname, name)
    if not elm:
        sys.stderr.write("Unable to create " + printedname + " \n")
    if detail:
        sys.stderr.write(detail)
    return elm

Next, we initialize GStreamer and create an empty pipeline.

In [ ]:
# Standard GStreamer initialization
Gst.init(None)

# Create gstreamer elements
# Create Pipeline element that will form a connection of other elements
print("Creating Pipeline \n")
pipeline = Gst.Pipeline()

if not pipeline:
    sys.stderr.write(" Unable to create Pipeline \n")

Then, we create the elements that are required for our pipeline.

In [ ]:
########### Create elements required for the Pipeline ###########
# Source element for reading from the file
source = make_elm_or_print_err("filesrc", "file-source", "Source")
# Since the data format in the input file is elementary h264 stream, we need a h264parser
h264parser = make_elm_or_print_err("h264parse", "h264-parser", "h264 parse")
# Use nvdec_h264 for hardware accelerated decode on GPU
decoder = make_elm_or_print_err("nvv4l2decoder", "nvv4l2-decoder", "Nvv4l2 Decoder")
# Create nvstreammux instance to form batches from one or more sources
streammux = make_elm_or_print_err("nvstreammux", "Stream-muxer", "NvStreamMux")
# Use nvinfer to run inferencing on decoder's output, behavior of inferencing is set through config file
pgie = make_elm_or_print_err("nvinfer", "primary-inference", "pgie")
# Use nvtracker to give objects unique-ids
tracker = make_elm_or_print_err("nvtracker", "tracker", "tracker")
# Use convertor to convert from NV12 to RGBA as required by nvosd
nvvidconv = make_elm_or_print_err("nvvideoconvert", "convertor", "nvvidconv")
# Create OSD to draw on the converted RGBA buffer
nvosd = make_elm_or_print_err("nvdsosd", "onscreendisplay", "nvosd")
# Finally encode and save the osd output
queue = make_elm_or_print_err("queue", "queue", "Queue")
# Use convertor to convert from NV12 to RGBA as required by nvosd
nvvidconv2 = make_elm_or_print_err("nvvideoconvert", "convertor2", "nvvidconv2")
# Place an encoder instead of OSD to save as video file
encoder = make_elm_or_print_err("avenc_mpeg4", "encoder", "Encoder")
# Parse output from Encoder
codeparser = make_elm_or_print_err("mpeg4videoparse", "mpeg4-parser", "Code Parser")
# Create a container
container = make_elm_or_print_err("qtmux", "qtmux", "Container")
# Create Sink for storing the output
sink = make_elm_or_print_err("filesink", "filesink", "Sink")

Now that we have created the elements, we can proceed to set various properties for our pipeline.

## Understanding the configuration files

We'll resuse the `pgie` configuration file that was examined in the previous notebook. If you haven't already set your API key in the configuration file [here](../source_code/N4/pgie_yolov4_tao_config.txt) in the field `tlt-model-key`, please go ahead and do so, then save the file with `ctrl s`. Not setting the key makes it impossible to decrypt the model and successfully run this notebook.

In [ ]:
print("Please replace the tlt-model-key variable with your key.")

In [ ]:
!cat ../source_code/N4/pgie_yolov4_tao_config.txt

We can also visualize the configuration file for our nvtracker (tracking plugin) named `dstest2_tracker_config.txt`. The configuration file is parsed and properties are then set for the tracker.

In [ ]:
!cat ../source_code/N4/dstest2_tracker_config.txt

In the next cell, we set the properties for the elements of our pipeline, including but not limited to the contents of the two configuration files.

In [ ]:
############ Set properties for the Elements ############
print("Playing file ", INPUT_VIDEO_NAME)
# Set Input File Name 
source.set_property("location", INPUT_VIDEO_NAME)
# Set Input Width, Height and Batch Size 
streammux.set_property("width", 1920)
streammux.set_property("height", 1080)
streammux.set_property("batch-size", 1)
# Timeout in microseconds to wait after the first buffer is available 
# to push the batch even if a complete batch is not formed.
streammux.set_property("batched-push-timeout", 4000000)
# Set Congifuration file for nvinfer 
pgie.set_property("config-file-path", "../source_code/N4/pgie_yolov4_tao_config.txt")
#Set properties of tracker from tracker_config
config = configparser.ConfigParser()
config.read("../source_code/N4/dstest2_tracker_config.txt")
config.sections()
for key in config['tracker']:
    if key == 'tracker-width' :
        tracker_width = config.getint('tracker', key)
        tracker.set_property('tracker-width', tracker_width)
    if key == 'tracker-height' :
        tracker_height = config.getint('tracker', key)
        tracker.set_property('tracker-height', tracker_height)
    if key == 'gpu-id' :
        tracker_gpu_id = config.getint('tracker', key)
        tracker.set_property('gpu_id', tracker_gpu_id)
    if key == 'll-lib-file' :
        tracker_ll_lib_file = config.get('tracker', key)
        tracker.set_property('ll-lib-file', tracker_ll_lib_file)
    if key == 'll-config-file' :
        tracker_ll_config_file = config.get('tracker', key)
        tracker.set_property('ll-config-file', tracker_ll_config_file)
    if key == 'enable-batch-process' :
        tracker_enable_batch_process = config.getint('tracker', key)
        tracker.set_property('enable_batch_process', tracker_enable_batch_process)
# Set Encoder bitrate for output video
encoder.set_property("bitrate", 2000000)
# Set Output file name and disable sync and async
sink.set_property("location", OUTPUT_VIDEO_NAME)
sink.set_property("sync", 0)
sink.set_property("async", 0)

We now link all the elements in the order we prefer and create Gstreamer bus to feed all messages through it.

In [ ]:
########## Add and Link Elements in the Pipeline ##########

print("Adding elements to Pipeline \n")

############# ~~~~~~~ COMPLETE THIS SECTION ~~~~~~~ #############
# Adding elements to the pipeline
pipeline.add(source)
pipeline.add(h264parser)
#
#
#
#
#
#
#
#
#
#
#
#
###################### ~~~~~~~ END ~~~~~~~ ######################

# We now  link the elements together 
# file-source -> h264-parser -> nvh264-decoder -> nvinfer -> nvvidconv ->
# queue -> nvvidconv2 -> encoder -> parser -> container -> sink -> output-file
print("Linking elements in the Pipeline \n")
source.link(h264parser)
h264parser.link(decoder)

##### Creating Sink pad and source pads and linking them together 

# Create Sinkpad to Streammux 
sinkpad = streammux.get_request_pad("sink_0")
if not sinkpad:
    sys.stderr.write(" Unable to get the sink pad of streammux \n")
# Create source pad from Decoder   
srcpad = decoder.get_static_pad("src")
if not srcpad:
    sys.stderr.write(" Unable to get source pad of decoder \n")

############# ~~~~~~~ COMPLETE THIS SECTION ~~~~~~~ #############
# Link the elements
srcpad.link(sinkpad)
streammux.link(pgie)
#
#
#
#
#
#
#
#
#
###################### ~~~~~~~ END ~~~~~~~ ######################

Now we create an event loop and feed GStreamer bus messages to it.

In [ ]:
loop = GLib.MainLoop()
bus = pipeline.get_bus()
bus.add_signal_watch()
bus.connect ("message", bus_call, loop)

## Working with the metadata 

Our pipeline now carries the metadata forward but does nothing with it up to this moment. As mentioned in the above pipeline diagram, we will now create a callback function to display relevant data on the frame once it is called and create a sink pad in the `nvosd` element to call the function.

In [ ]:
############## Working with the Metadata ################

def osd_sink_pad_buffer_probe(pad, info, u_data):
    
    ############# ~~~~~~~ COMPLETE THIS SECTION ~~~~~~~ #############
    # Intiallizing object counter with 0
    obj_counter = {
        PGIE_CLASS_ID_FRESHAPPLE:0,
        #
        #
        #
        #
        #
    }
    ###################### ~~~~~~~ END ~~~~~~~ ######################
    
    # Colors of the bounding boxes in RGBA
    obj_colors = {
        PGIE_CLASS_ID_FRESHAPPLE:(1.0, 0.0, 0.0, 0.0),
        PGIE_CLASS_ID_FRESHBANANA:(0.0, 1.0, 0.0, 0.0),
        PGIE_CLASS_ID_FRESHORANGE:(0.0, 0.0, 1.0, 0.0),
        PGIE_CLASS_ID_ROTTENAPPLE:(0.0, 1.0, 1.0, 0.0),
        PGIE_CLASS_ID_ROTTENBANANA:(1.0, 0.0, 1.0, 0.0),
        PGIE_CLASS_ID_ROTTENORANGE:(1.0, 1.0, 0.0, 0.0)
    }
    # Set frame_number & rectangles to draw as 0 
    frame_number=0
    num_rects=0
    
    gst_buffer = info.get_buffer()
    if not gst_buffer:
        print("Unable to get GstBuffer ")
        return

    # Retrieve batch metadata from the gst_buffer
    # Note that pyds.gst_buffer_get_nvds_batch_meta() expects the
    # C address of gst_buffer as input, which is obtained with hash(gst_buffer)
    batch_meta = pyds.gst_buffer_get_nvds_batch_meta(hash(gst_buffer))
    l_frame = batch_meta.frame_meta_list
    
    while l_frame is not None:
        try:
            # Note that l_frame.data needs a cast to pyds.NvDsFrameMeta
            frame_meta = pyds.NvDsFrameMeta.cast(l_frame.data)
        except StopIteration:
            break
        
        # Get frame number, number of rectangles to draw and object metadata
        frame_number=frame_meta.frame_num
        num_rects = frame_meta.num_obj_meta
        l_obj=frame_meta.obj_meta_list
        
        while l_obj is not None:
            try:
                # Casting l_obj.data to pyds.NvDsObjectMeta
                obj_meta=pyds.NvDsObjectMeta.cast(l_obj.data)
            except StopIteration:
                break
            # Increment object class by 1 and set box border color  
            obj_counter[obj_meta.class_id] += 1
            r, g, b, a = obj_colors[obj_meta.class_id]
            obj_meta.rect_params.border_color.set(r, g, b, a)
            try: 
                l_obj=l_obj.next
            except StopIteration:
                break
        ################## Setting Metadata Display configruation ############### 
        # Acquiring a display meta object
        display_meta=pyds.nvds_acquire_display_meta_from_pool(batch_meta)
        display_meta.num_labels = 1
        py_nvosd_text_params = display_meta.text_params[0]
        # Setting display text to be shown on screen
        py_nvosd_text_params.display_text = "Frame Number={} Number of Objects={} Freshapple_count={} Freshbanana_count={} " \
            "Freshorange_count={} Rottenapple_count={} Rottenbanana_count={} Rottenorange_count={}".format(frame_number, num_rects, 
            obj_counter[PGIE_CLASS_ID_FRESHAPPLE], obj_counter[PGIE_CLASS_ID_FRESHBANANA], obj_counter[PGIE_CLASS_ID_FRESHORANGE], 
            obj_counter[PGIE_CLASS_ID_ROTTENAPPLE], obj_counter[PGIE_CLASS_ID_ROTTENBANANA], obj_counter[PGIE_CLASS_ID_ROTTENORANGE])
        
        # Now set the offsets where the string should appear
        py_nvosd_text_params.x_offset = 10
        py_nvosd_text_params.y_offset = 12
        # Font, font-color and font-size
        py_nvosd_text_params.font_params.font_name = "Serif"
        py_nvosd_text_params.font_params.font_size = 14
        # Set(red, green, blue, alpha); Set to White
        py_nvosd_text_params.font_params.font_color.set(1.0, 1.0, 1.0, 1.0)
        # Text background color
        py_nvosd_text_params.set_bg_clr = 1
        # Set(red, green, blue, alpha); set to Black
        py_nvosd_text_params.text_bg_clr.set(0.0, 0.0, 0.0, 1.0)
        # Using pyds.get_string() to get display_text as string to print in notebook
        print(pyds.get_string(py_nvosd_text_params.display_text))
        pyds.nvds_add_display_meta_to_frame(frame_meta, display_meta)
        
        ############################################################################
        
        try:
            l_frame=l_frame.next
        except StopIteration:
            break
    return Gst.PadProbeReturn.OK

Here we add the probe to get informed of the meta data generated. We add probe to the sink pad of the osd element, since by that time, the buffer would have got all the metadata.

In [ ]:
osdsinkpad = nvosd.get_static_pad("sink")
if not osdsinkpad:
    sys.stderr.write(" Unable to get sink pad of nvosd \n")
    
osdsinkpad.add_probe(Gst.PadProbeType.BUFFER, osd_sink_pad_buffer_probe, 0)

## Run the pipeline

Now with everything defined, we can start the playback and listen to the events.

In [ ]:
# start play back and listen to events
print("Starting pipeline \n")
start_time = time.time()
pipeline.set_state(Gst.State.PLAYING)
try:
    loop.run()
except:
    pass
# cleanup
pipeline.set_state(Gst.State.NULL)
print("--- %s seconds ---" % (time.time() - start_time))

With the next cell, we convert the video profile to be compatible with Jupyter notebook.

In [ ]:
!ffmpeg -loglevel panic -y -an -i ../source_code/challenge_deepstream/ds_out.mp4 -vcodec libx264 -pix_fmt yuv420p -profile:v baseline -level 3 ../source_code/challenge_deepstream/output.mp4

In [ ]:
# Display the output
from IPython.display import HTML
HTML("""
 <video width="640" height="480" controls>
 <source src="../source_code/challenge_deepstream/output.mp4"
 </video>
""".format())

In this notebook, you have reviewed the deployment with DeepStream. In the next one, you will practice deployment with Triton Inference Server. You will need to reactivate the Triton environment and the server container in order to complete the second challenge notebook. Please check the `README` file on how to do so.

***

## Licensing

This material is released by OpenACC-Standard.org, in collaboration with NVIDIA Corporation, under the Creative Commons Attribution 4.0 International (CC BY 4.0).

<br>
<div>
    <span style="float: left; width: 33%; text-align: left;"><a href="5.Measure_object_size_using_OpenCV.ipynb">Previous Notebook</a></span>
    <span style="float: left; width: 34%; text-align: center;">
        <a href="1.Data_labeling_and_preprocessing.ipynb">1</a>
        <a href="2.Object_detection_using_TAO_YOLOv4.ipynb">2</a>
        <a href="3.Model_deployment_with_Triton_Inference_Server.ipynb">3</a>
        <a href="4.Model_deployment_with_DeepStream.ipynb">4</a>
        <a href="5.Measure_object_size_using_OpenCV.ipynb">5</a>
        <a >6</a>
        <a href="7.Challenge_Triton.ipynb">7</a>
    </span>
    <span style="float: left; width: 33%; text-align: right;"><a href="7.Challenge_Triton.ipynb">Next Notebook</a></span>
</div>

<br>
<p> <center> <a href="../Start_here.ipynb">Home Page</a> </center> </p>